In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wrangle as w
import explore as e
import model as m

from scipy.stats import ttest_1samp
from sklearn.metrics import mean_squared_error
from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

%matplotlib inline

In [2]:
df = w.wrangle_new_york()
train, val, test = w.split_new_york(df)
X_train, y_train, X_val, y_val, X_test, y_test = m.X_y_modeling_split(train, val, test, 'avg_temp')


In [3]:
yhat_df = m.rolling_avg_model(train, val)

In [4]:
def evaluate(target_var):
    rmse = round(sqrt(mean_squared_error(y_val[target_var], yhat_df[target_var])), 0)
    return rmse

In [5]:
evaluate('avg_temp')

TypeError: 'NoneType' object is not subscriptable